In [ ]:
#Importing the Libraries
import pandas as pd
import joblib
import requests
import warnings
from datetime import datetime
warnings.filterwarnings('ignore', category=UserWarning)

/Users/shashanksingh/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/shashanksingh/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [ ]:
#Data Loading
fuel_df = pd.read_csv('fuel_prices.csv')
fuel_df.head()

,Date,City,Petrol_Price,Diesel_Price
0,2025-10-22,Chennai,100.80,92.39
1,2025-10-22,Delhi,94.77,87.67
2,2025-10-22,Kolkata,105.41,92.02
3,2025-10-22,Mumbai,103.50,90.03
4,2025-10-21,Chennai,100.80,92.39


In [3]:
def get_fuel_price(fuel_df, city, date):
    try:
        user_date = pd.to_datetime(date)
        local_fuel_df = fuel_df.copy()
        local_fuel_df['Date'] = pd.to_datetime(local_fuel_df['Date'])
        city_data = local_fuel_df[local_fuel_df['City'] == city].sort_values(by='Date')
        
        if city_data.empty:
            print(f"Warning: No fuel price data found for city: {city}.")
            return None, None, None
        exact_match = city_data[city_data['Date'] == user_date]
        
        if not exact_match.empty:
            row = exact_match.iloc[0]
            print(f"Fuel Price: Found exact match for {date}.")
            return row['Petrol_Price'], row['Diesel_Price'], row['Date']
        else:
            fallback_data = city_data[city_data['Date'] <= user_date]
            
            if not fallback_data.empty:
                row = fallback_data.iloc[-1]
                print(f"Fuel Price: No exact match for {date}. Using most recent data from {row['Date'].strftime('%Y-%m-%d')}.")
                return row['Petrol_Price'], row['Diesel_Price'], row['Date']
            else:
                row = city_data.iloc[0]
                print(f"Fuel Price: Date {date} is older than our data. Using oldest data from {row['Date'].strftime('%Y-%m-%d')}.")
                return row['Petrol_Price'], row['Diesel_Price'], row['Date']

    except Exception as e:
        print(f"Error in get_fuel_prices: {e}")
        return None, None, None

In [4]:
CITY_COORDS = {
    'Delhi': {'lat': 28.61, 'lon': 77.21},
    'Mumbai': {'lat': 19.08, 'lon': 72.88},
    'Chennai': {'lat': 13.08, 'lon': 80.27},
    'Kolkata': {'lat': 22.57, 'lon': 88.36}
}

In [ ]:
#This function makes a live API call
def get_weather_forecast(city, date):
    """
    Gets the forecasted weather for a specific date from Open-Meteo.
    """
    if city not in CITY_COORDS:
        print(f"Warning: No coordinates for city: {city}.")
        return None, None
        
    coords = CITY_COORDS[city]
    api_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        'latitude': coords['lat'],
        'longitude': coords['lon'],
        'daily': 'temperature_2m_mean,precipitation_sum',
        'start_date': date,
        'end_date': date,
        'timezone': 'auto'
    }
    
    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()
        results = response.json()
        
        temp = results['daily']['temperature_2m_mean'][0]
        precip = results['daily']['precipitation_sum'][0]
        print(f"Weather: Successfully fetched forecast for {date}.")
        return temp, precip
    except Exception as e:
        print(f"Warning: Could not fetch weather. {e}")
        return None, None

In [ ]:
#User Inputs 
user_date = '2025-10-22'
user_city = 'Mumbai'
user_vehicle = 'Truck'
user_route = 'Route_A'
user_distance = 50
user_load = 5000

budget_threshold = 5000

In [ ]:
#Model and Data Loading
model_cost = joblib.load("Regression_model.pkl")
model_risk = joblib.load("Classification-Model.pkl")
final_df = pd.read_csv('final_dataset.csv')

In [ ]:
#Data Fetching
petrol_price, diesel_price, fuel_date_used = get_fuel_price(fuel_df, user_city, user_date)
temp, precip = get_weather_forecast(user_city, user_date)
if petrol_price is None or temp is None:
    print("Error: Could not fetch required data. Aborting prediction.")
    exit()

Fuel Price: Found exact match for 2025-10-22.
Weather: Successfully fetched forecast for 2025-10-22.


In [ ]:
#Data Preprocessing for Models
fuel_type = 'Diesel' if user_vehicle in ['Truck', 'Van'] else 'Petrol'
new_trip_data = {
    'City': user_city,
    'Vehicle_Type': user_vehicle,
    'Fuel_Type': fuel_type,
    'Route': user_route,
    'Distance_km': user_distance,
    'Load_Weight_kg': user_load,
    'Petrol_Price': petrol_price,
    'Diesel_Price': diesel_price,
    'Temp_Mean_C': temp,
    'Precipitation_mm': precip
}

In [10]:
columns = [
    'City', 'Vehicle_Type', 'Fuel_Type', 'Route', 'Distance_km', 
    'Load_Weight_kg', 'Petrol_Price', 'Diesel_Price', 'Temp_Mean_C', 
    'Precipitation_mm'
]
new_trip_df = pd.DataFrame([new_trip_data], columns=columns)
print(new_trip_df.T)

                        0
City               Mumbai
Vehicle_Type        Truck
Fuel_Type          Diesel
Route             Route_A
Distance_km            50
Load_Weight_kg       5000
Petrol_Price        103.5
Diesel_Price        90.03
Temp_Mean_C          28.9
Precipitation_mm      0.0


In [ ]:
#Prediction Step
predicted_cost = model_cost.predict(new_trip_df)[0]
prediction_risk = model_risk.predict(new_trip_df)[0]
probabilities_risk = model_risk.predict_proba(new_trip_df)[0]
prob_over_budget = probabilities_risk[1]

In [12]:
print("\n--- 📊 PREDICTION REPORT ---")
print("==========================================")
print(f"  Model 1 (Cost Prediction):")
print(f"  Predicted Trip Cost:   ₹{predicted_cost:.2f}")
print("------------------------------------------")
print(f"  Model 2 (Risk Analysis):")
print(f"  Budget Threshold:      ₹{budget_threshold}")
print(f"  Probability Over Budget: {prob_over_budget * 100:.1f}%")
print(f"  Final Decision:        {'FLAGGED: LIKELY OVER BUDGET' if prediction_risk == 1 else 'APPROVED: LIKELY UNDER BUDGET'}")
print("==========================================")


--- 📊 PREDICTION REPORT ---
  Model 1 (Cost Prediction):
  Predicted Trip Cost:   ₹1022.08
------------------------------------------
  Model 2 (Risk Analysis):
  Budget Threshold:      ₹5000
  Probability Over Budget: 37.0%
  Final Decision:        APPROVED: LIKELY UNDER BUDGET
